<a href="https://colab.research.google.com/github/dowrave/Tensorflow_Basic/blob/main/220517_tf_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tf.data
- `tf.data` API로 간단하고 재사용 가능한 조각으로 입력 파이프라인을 빌드할 수 있다.
  - 예를 들어 이미지 모델의 파이프라인은 분산된 파일 시스템의 파일에서 데이터 집계, 각 이미지에 임의의 Perturbation 적용, 무작위로 선택한 이미지를 학습을 위한 batch로 병합할 수 있음.
  - 텍스트 모델의 경우 원시 텍스트 데이터에서 심볼 추출, 룩업 테이블이 있는 embedding 식별자로 변환, 길이가 서로 다른 시퀀스를 batch 처리하는 과정을 포함할 수 있다.

- `tf.data` API는 `tf.data.Dataset` 추상화를 도입하며, 각 요소는 하나 이상의 구성요소로 이루어진다. 예를 들어 이미지라면 (이미지, 해당 label)인 것.

- 데이터 세트 생성 방법 
1. 데이터 소스 : 메모리 또는 하나 이상의 파일에 저장된 데이터에서 데이터셋 구성
2. 데이터 변환 : `tf.data.Dataset` 객체에서 데이터셋 구성

In [ ]:
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.set_printoptions(precision = 4) # 소수점 4번째 자리까지?

## 기본 매커니즘
- `데이터 소스`로 시작 
- 메모리의 데이터에서 `Dataset`를 구성하겠다면 `tf.data.Dataset.from_tensors()` 또는 `tf.data.Dataset.from_tensor_slices()`을 쓸 수 있다. 
- TFRecord로 저장되었다면 `tf.data.TFRecordDataset()`을 쓸 수도 있다.

- `Dataset` 객체는 메서드 호출로 새로운 `Dataset`로 변환할 수 있다. 
  - 요소별 변환 : `Dataset.map()`
  - 다중 요소 변환 : `Dataset.batch()`

- 파이썬 반복문도 돌릴 수 있음

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([8, 3, 0, 8, 2, 1])
dataset

In [ ]:
for elem in dataset:
  print(elem.numpy())

In [ ]:
it = iter(dataset) # 파이썬 반복기 생성해서

# 1개씩 뽑을 수도 있음
print(next(it).numpy()) 
print(next(it).numpy()) 

In [ ]:
# reduce를 사용해 데이터세트 요소들을 소비할 수도 있다.
print(dataset.reduce(0, lambda state, value : state + value).numpy())

## 데이터세트 구조
- 데이터세트는 각 요소가 동일한 구성 요소(중첩) 구조를 갖는 요소의 시퀀스를 생성한다.
- 개별 구성요소는 `tf.TypeSpec`에 표현되는 유형일 수 있는데, `tf.Tensor`, `tf.TensorArray`, `tf.data.Dataset` 등도 포함된다.
<br>
- 파이썬 구조에는 `tuple`, `dict`, `NamedTuple`, `OrderedDict`가 있다.
- `list`는 데이터세트 요소의 구조를 표현하는 데 유효하지 않다. `list`는 `tuple`로 변환해야 구조로 처리할 수 있다. `list` 출력을 단일 구성 요소로 쓰고 싶다면 `tf.stack`을 사용해 이를 명시적으로 구성해야 한다.
<br>
- `Dataset.element_spec` 속성으로 각 요소의 구성 요소 유형을 검사할 수 있다.

In [ ]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4, 10]))
dataset1.element_spec

In [ ]:
dataset2 = tf.data.Dataset.from_tensor_slices(
    (tf.random.uniform([4]),
     tf.random.uniform([4, 100], maxval = 100, dtype =tf.int32))
)

dataset2.element_spec

In [ ]:
# zip 함수로 묶을 수 있음
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

dataset3.element_spec

In [ ]:
# sparse 텐서도 포함 가능
dataset4 = tf.data.Dataset.from_tensors(tf.SparseTensor(indices = [[0, 0], [1, 2]], values = [1, 2], dense_shape = [3,4]))

dataset4.element_spec

In [ ]:
# value_type으로 대표되는 데이터 타입을 볼 수도 있다.
dataset4.element_spec.value_type

`Dataset` 변환은 모든 구조의 데이터세트를 지원한다.

In [ ]:
dataset1 = tf.data.Dataset.from_tensor_slices(
    tf.random.uniform([4, 10], minval = 1, maxval = 10, dtype = tf.int32)
)
dataset1

In [ ]:
dataset2 = tf.data.Dataset.from_tensor_slices(
    (tf.random.uniform([4]),
     tf.random.uniform([4, 100], maxval = 100, dtype = tf.int32))
)

In [ ]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))
dataset3

In [ ]:
for a, (b, c) in dataset3:
  print(a.shape, b.shape, c.shape)

## 입력 데이터 읽기
### Numpy Array
- 모든 입력 데이터가 메모리에 맞다면 `Tensor` 객체로 변환 후 `Dataset.from_tensor_slices()`를 이용한다.


In [ ]:
train, test = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
images, labels = train
images = images/255.

dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset

- 위 방법은 `feature, label` 배열을 `tf.constant()` 연산으로 tf 그래프에 임베딩 처리한다. 배열의 내용이 여러 번 복사되기 때문에 `tf.GraphDef` 프로토콜 버퍼의 2GB 제한에 걸릴 수 있음

### Python Generator
- 편리하지만 이식성, 확장성이 제한적임.
- 생성기를 생성한 것과 동일한 파이썬 프로세스에서 실행되어야 하며 여전히 파이썬 GIL의 영향을 받는다.


In [ ]:
def count(stop):
  i = 0
  while i < stop:
    yield i # yield : generator object를 반환함
            # generator : 데이터를 미리 만들지 않고 필요할 때 하나씩 만들어내는 객체
    i += 1

#### 잠깐! yield와 return 차이에 대해서 짚고 넘어감
[차이점]
<br>
1. 아래 함수에서 return은 리스트를, yield는 generator를 반환함.
2. `return`은 모든 결과값을 메모리에 올려야 하지만, `yield`는 결과값을 하나씩 메모리에 올려놓음 - `lazy iterator`라고도 함. 메모리 관리 측면에서 yield가 더 효율적임
  - 이런 특성이 있기 때문에 데이터를 무한히 뱉는 함수는 `yield`가 꼭 필요함(무한 while문에서 return을 잡으면 그게 다 메모리로 잡히지만, yield로 잡으면 1개씩 뱉으니까)

[리스트 -> 제너레이터 변환]
<br>
- return 대신 yield from 쓰면 됨

[generator comprehension]
<br>
- [] 대신 ()쓰면 됨. 사용법은 동일

In [ ]:
import time

def return_abc():
  alphabets = []
  for ch in "ABC":
    time.sleep(1)
    alphabets.append(ch)
  return alphabets

def yield_abc():
  for ch in "ABC":
    time.sleep(1)
    yield ch

# 3초 후 abc가 한꺼번에 출력됨
for ch in return_abc():
  print(ch) 

# 1초 후 a, 1초 후 b, 1초 후 c가 출력됨.
for ch in yield_abc():
  print(ch)

In [ ]:
for n in count(5):
  print(n)

- 이렇게 파이썬으로 생성된 생성자를 `Dataset.from_generator`로 받을 수 있음.
- 이 생성자는 callable을 입력으로 받으며, 생성기가 끝에 도달하면 다시 시작할 수 있다.

In [ ]:
ds_counter = tf.data.Dataset.from_generator(count, args = [25], output_types = tf.int32, output_shapes = (),)
for count_batch in ds_counter.repeat().batch(10).take(10):
  print(count_batch.numpy())

`output_shapes` : 많은 텐서플로우 연산이 모르는 랭크의 텐서를 지원하지 않기 때문에 쓰는 걸 권장함. 축의 길이를 모르거나 가변적이라면 해당 값을 `None`으로 설정할 것.

In [ ]:
def gen_series():
  i = 0
  while True:
    size = np.random.randint(0, 10)
    yield i, np.random.normal(size = (size, ))
    i += 1

In [ ]:
for i, series in gen_series():
  print(i, " : ", str(series))
  if i > 5:
    break

In [ ]:
ds_series = tf.data.Dataset.from_generator(
    gen_series,
    output_types = (tf.int32, tf.float32),
    output_shapes = ((), (None,)) # 2번째 항목이 알 수 없는 길이 None으로 전달됨
)

ds_series

- 가변 형상의 데이터세트 배치 처리는 `Dataset.padded_batch`를 사용한다.

In [ ]:
ds_series_batch = ds_series.shuffle(20).padded_batch(10) 

ids, sequence_batch = next(iter(ds_series_batch))
print(ids.numpy())
print()
print(sequence_batch.numpy())

### 더 현실적인 예시
`preprocessing.image.ImageDataGenerator`를 `tf.data.Dataset`으로 래핑하기

In [ ]:
flowers = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar = True
)

In [ ]:
# ImageDataGenerator 만들기
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1. / 255, rotation_range = 20)

images, labels = next(img_gen.flow_from_directory(flowers))

In [ ]:
print(images.dtype, images.shape)
print(labels.dtype, labels.shape)

In [ ]:
ds = tf.data.Dataset.from_generator(
    lambda : img_gen.flow_from_directory(flowers), 
    output_types = (tf.float32, tf.float32), 
    output_shapes = ([32, 256, 256, 3], [32, 5])
    )

ds.element_spec

In [ ]:
for images, label in ds.take(1):
  print(images.shape, labels.shape)

### TFRecord 데이터 소비하기
- `tf.data` API는 메모리에 맞지 않는 큰 데이터세트를 처리할 수 있다.
- `tf.data.TFRecordDataset` 클래스로 하나 이상의 TFRecord 파일 내용을 스트리밍할 수 있다.

In [ ]:
fsns_test_file = tf.keras.utils.get_file("fsns.tfrec",
                                         "https://storage.googleapis.com/download.tensorflow.org/data/fsns-20160927/testdata/fsns-00000-of-00001")


- `TFRecordDataset`의 `filenames`는 문자열, 문자열 목록, `tf.Tensor` 문자열일 수 있음.
- 학습, 검증 목적으로 두 파일 세트를 사용한다면 파일 이름을 입력 인수로 써서 데이터 세트를 생성하는 팩토리 메소드를 작성할 수 있다.

In [ ]:
dataset = tf.data.TFRecordDataset(filenames = [fsns_test_file])
dataset

In [ ]:
raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())

parsed.features.feature['image/text']

### 텍스트 데이터 소비하기

In [ ]:
directory_url = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
filenames = ['cowper.txt', 'derby.txt', 'butler.txt'] # 여러 파일을 가져올 수 있음

file_paths = [
              tf.keras.utils.get_file(file_name, directory_url + file_name) for file_name in filenames
]

In [ ]:
dataset = tf.data.TextLineDataset(file_paths)

In [ ]:
for line in dataset.take(5):
  print(line.numpy())

In [ ]:
# 파일 사이에서 줄 바꾸기
files_ds = tf.data.Dataset.from_tensor_slices(file_paths)
lines_ds = files_ds.interleave(tf.data.TextLineDataset, cycle_length = 3) # 이걸로 파일을 쉽게 섞을 수 있다.

for i, line in enumerate(lines_ds.take(9)):
  if i % 3 == 0:
    print()
  print(line.numpy())

- `TextLineDataset`은 각 파일의 모든 줄을 생성한다 : 헤더 줄로 시작하거나 주석이 포함되었다면 바람직하지 않을 수 있음.
  - 이러한 줄은 `Dataset.skip()` 또는 `Dataset.filter()` 변환으로 제거할 수 있다.


In [ ]:
titanic_file = tf.keras.utils.get_file('train.csv', 'https://storage.googleapis.com/tf-datasets/titanic/train.csv')
titanic_lines = tf.data.TextLineDataset(titanic_file)

In [ ]:
for line in titanic_lines.take(10):
  print(line.numpy())

In [ ]:
def survived(line):
  return tf.not_equal(tf.strings.substr(line, 0, 1), "0")

survivors = titanic_lines.skip(1).filter(survived)

In [ ]:
for line in survivors.take(10):
  print(line.numpy())

### CSV 데이터 소비하기

In [ ]:
titanic_file = tf.keras.utils.get_file('train.csv', 'https://storage.googleapis.com/tf-datasets/titanic/train.csv')
df = pd.read_csv(titanic_file)
df.head()

`Dataset.from_tensor_slices` : 데이터가 메모리에 맞다면 사용가능

In [ ]:
titanic_slices = tf.data.Dataset.from_tensor_slices(dict(df))

for feature_batch in titanic_slices.take(1):
  for key, value in feature_batch.items():
    print("{!r:20s} : {}".format(key, value))

- 더 확장성 있는 방식은 필요에 따라 디스크에서 로드하는 것이다.
- `tf.data`는 RFC 4180을 준수하는 하나 이상의 CSV 파일에서 레코드를 추출하는 메소드를 제공한다.
- `experimental.make_csv_dataset` 함수는 csv 파일 세트를 읽는 고급 인터페이스로, 열 형식 유추를 비롯해 배치, 셔플 등 많은 기능을 지원함

In [ ]:
titanic_batches = tf.data.experimental.make_csv_dataset(titanic_file, batch_size = 4, label_name = 'survived')

In [ ]:
for feature_batch, label_batch in titanic_batches.take(1):
  print("Survived : {}".format(label_batch))
  print("Features : ")
  for key, value in feature_batch.items():
    print("{!r:20s}: {}".format(key, value))


In [ ]:
# 열의 일부만 필요하다면 select_columns 인수를 이용한다.
titanic_batches = tf.data.experimental.make_csv_dataset(
    titanic_file, batch_size = 4,
    label_name = 'survived', select_columns = ['class', 'fare', 'survived']
)

for feature_batch, label_batch in titanic_batches.take(1):
  print("'survived' : {}".format(label_batch))
  for key, value in feature_batch.items():
    print(" {!r:20s} : {}".format(key, value))

- 더 세밀한 제어를 지원하는 `experimental.CsvDataset` 클래스도 있다. 열 형식 유추를 지원하지 않고, 각 열의 유형을 지원해야 함. 귀찮으니 생략.
- 헤더 제거 기능은 지원한다. `header` 혹은 `select_cols`가 있음.

### 파일 세트 소비하기

In [ ]:
flowers_root = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar = True
)
flowers_root = pathlib.Path(flowers_root)

for item in flowers_root.glob('*'):
  print(item.name)

In [ ]:
list_ds = tf.data.Dataset.list_files(str(flowers_root/'*/*'))

for f in list_ds.take(5):
  print(f.numpy())

In [ ]:
def process_path(file_path):
  label = tf.strings.split(file_path, os.sep)[-2]
  return tf.io.read_file(file_path), label

labeled_ds = list_ds.map(process_path)

In [ ]:
for image_raw, label_text in labeled_ds.take(1):
  print(repr(image_raw.numpy()[:100]))
  print()
  print(label_text.numpy())

## 데이터 요소 배치 처리

### 간단한 배치 처리
- `Dataset.batch()`

In [ ]:
inc_dataset = tf.data.Dataset.range(100)
dec_dataset = tf.data.Dataset.range(0, -100, -1)
dataset = tf.data.Dataset.zip((inc_dataset, dec_dataset))
batched_dataset = dataset.batch(4)

for batch in batched_dataset.take(4):
  print([arr.numpy() for arr in batch])

In [ ]:
batched_dataset # shape = None - 마지막 배치가 완전히 채워지지 않을 가능성이 있어서 None(동적 가능)으로 둠

In [ ]:
batched_dataset = dataset.batch(7, drop_remainder = True) # 꽉 채워지지 않는 마지막 배치는 무시함
batched_dataset # shape가 7로 채워진 것을 볼 수 있음

### 패딩이 있는 텐서 배치 처리
- 많은 모델은 다양한 크기를 가지는 입력 데이터로 작동한다.
- 이 때 `Dataset.padded_batch` 변환으로 패딩 처리될 수 있는 하나 이상의 차원을 지정해 서로 다른 형상의 텐서를 배치 처리할 수 있다.

In [ ]:
dataset = tf.data.Dataset.range(100)
dataset = dataset.map(lambda x : tf.fill([tf.cast(x, tf.int32)], x)) # [[], [1], [2, 2], [3, 3, 3]], ...
dataset = dataset.padded_batch(4, padded_shapes = (None,)) # batch_size = 4로 쪼갬 / shape = None은 가변 길이를 의미함

for batch in dataset.take(2):
  print(batch.numpy())
  print()

## 학습 워크 플로우

### 여러 에포크 처리하기
- 데이터 세트를 반복하는 가장 간단한 방법은 `Dataset.repeat()` 변환을 사용하는 것이다.

In [ ]:
titanic_file = tf.keras.utils.get_file('train.csv', 'https://storage.googleapis.com/tf-datasets/titanic/train.csv')
titanic_lines = tf.data.TextLineDataset(titanic_file)

In [ ]:
def plot_batch_sizes(ds):
  batch_sizes = [batch.shape[0] for batch in ds]
  plt.bar(range(len(batch_sizes)), batch_sizes)
  plt.xlabel('Batch Number')
  plt.ylabel('Batch Size')

`Dataset.repeat` : 인수가 없다면 무한
- 한 에포크의 끝과 다음 에포크의 시작을 알리지 않고 인수를 연결한다.
- 즉 `Dataset.batch` 후의 `Dataset.repeat`는 에포크 경계 양쪽에 걸쳐진 배치가 생성된다.

In [ ]:
titanic_batches = titanic_lines.repeat(3).batch(128)
plot_batch_sizes(titanic_batches)

- 만약 에포크 분리가 필요하다면, 반복 전에 `Dataset.batch`를 넣자.

In [ ]:
titanic_batches = titanic_lines.batch(128).repeat(3)

plot_batch_sizes(titanic_batches)

- 에포크의 끝에서 사용자 정의 계산(통계 수집 등)을 실행하려면 각 에포크에서 데이터 세트 반복을 다시 시작하는 게 가장 간단하다

In [ ]:
epochs = 3
dataset = titanic_lines.batch(128)

for epoch in range(epochs):
  for batch in dataset:
    print(batch.shape)

  print("End of Epoch : ", epoch)

### 입력 데이터의 임의 셔플
- `Dataset.shuffle()` 변환은 고정 크기 버퍼 유지 & 해당 버퍼에서 다음 요소 무작위로 균일하게 선택함
- buffer_sizes를 크게 가져간다면 더 철저하게 셔플되나 더 많은 메모리와 시간이 걸릴 수 있다. 문제가 된다면 파일 전체에서 `Dataset.interleave`를 사용하자.

In [ ]:
lines = tf.data.TextLineDataset(titanic_file)
counter = tf.data.experimental.Counter()

dataset = tf.data.Dataset.zip((counter, lines))
dataset = dataset.shuffle(buffer_size = 100)
dataset = dataset.batch(20)
dataset

In [ ]:
n, line_batch = next(iter(dataset))
print(n.numpy())
# buffer_size가 100, batch_size가 20이므로 1번째 배치에 120 이상의 인덱스를 가진 요소가 없다.
#  넣고 섞는거니까 100 이상이 없어야 하는 거 아니냐?

`Dataset.shuffle`은 버퍼가 비워질 때까지 에포크의 끝을 알리지 않는다.


In [ ]:
dataset = tf.data.Dataset.zip((counter, lines))
# 1. 셔플 -> repeat
shuffled = dataset.shuffle(buffer_size = 100).batch(10).repeat(2)

for n, line_batch in shuffled.skip(60).take(5):
  print(n.numpy())

In [ ]:
shuffle_repeat = [n.numpy().mean() for n, line_batch in shuffled]
plt.plot(shuffle_repeat, label="shuffle().repeat()")
plt.ylabel("Mean Item ID")
plt.legend()

In [ ]:
# 반복이 셔플 앞에 있으면 epoch 경계가 섞인다.
# 2. repeat -> shuffle
dataset = tf.data.Dataset.zip((counter, lines))
shuffled = dataset.repeat(2).shuffle(buffer_size = 100).batch(10)

for n, line_batch in shuffled.skip(55).take(15):
  print(n.numpy())

In [ ]:
repeat_shuffle = [n.numpy().mean() for n, line_batch in shuffled]

plt.plot(shuffle_repeat, label = 'shuffle().repeat()')
plt.plot(repeat_shuffle, label="repeat().shuffle()")
plt.ylabel('Mean Item ID')
plt.legend()

## 데이터 전처리하기
- `Dataset.map(f)` 변환은 f를 입력 데이터세트의 각 요소에 적용한다.
- `Dataset.map()` 사용법에 대한 일반적인 예를 다룬다.

In [ ]:
list_ds = tf.data.Dataset.list_files(str(flowers_root/
                                         '*/*'))

In [ ]:
# 데이터세트 요소 조작 함수
def parse_image(filename):
  parts = tf.strings.split(filename, os.sep)
  label = parts[-2]

  # 거의 고정된 레파토리임 - 정규화는 하지 않은 것에 주목
  image = tf.io.read_file(filename)
  image = tf.image.decode_jpeg(image) 
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, [128, 128])

  return image, label

In [ ]:
file_path = next(iter(list_ds))
image, label = parse_image(file_path)

def show(image, label):
  plt.figure()
  plt.imshow(image)
  plt.title(label.numpy().decode('utf-8'))
  plt.axis('off')

show(image, label)

In [ ]:
# 데이터세트에 매핑하기
images_ds = list_ds.map(parse_image)

for image, label in images_ds.take(2):
  show(image, label)

### 임의의 파이썬 로직 적용하기
- 가능한 텐서플로우의 작업을 사용하는 걸 권유함.
- 그러나 때로는 파이썬 라이브러리를 호출하는 게 유용한 경우가 있다. `Dataset.map()` 변환에서 `tf.py_function()` 연산을 사용할 수 있다.
  - 예를 들면 임의의 회전을 적용할 때 `tf.image` 모듈에는 `tf.image.rot90`만 있기 때문에 확대에는 유용하지 않음.
  - 참고 : `tensorflow_addons`에는 `tensorflow_addons.image.rotate`에서 호환되는 `rotate`가 있다.
  

In [ ]:
import scipy.ndimage as ndimage

def random_rotate_image(image):
  image = ndimage.rotate(image, np.random.uniform(-30, 30), reshape = False)
  return image


In [ ]:
image, label = next(iter(images_ds))
image = random_rotate_image(image)
show(image, label)

- 위에서 정의한 함수를 `Dataset.map`과 함께 쓴다면 함수를 적용할 때 반환 형상과 유형도 설명해야 한다.


In [ ]:
def tf_random_rotate_image(image, label):
  im_shape = image.shape
  [image, ] = tf.py_function(random_rotate_image, [image], [tf.float32]) # 설띵충
  image.set_shape(im_shape)
  return image, label

rot_ds = images_ds.map(tf_random_rotate_image)

for image,label in rot_ds.take(2):
  show(image, label)

### tf.Example 프로토콜 버퍼 메시지 구문 분석
- 많은 입력 파이프라인이 TFRecord 형식에서 `tf.train.Example` 프로토콜 버퍼 메시지를 추출함
- 각 `tf.train.Example` 레코드에는 하나 이상의 기능이 포함되며, 입력 파이프라인은 일반적으로 이러한 기능을 텐서로 변환함

In [ ]:
fsns_test_file = tf.keras.utils.get_file('fsns.tfrec', 'https://storage.googleapis.com/download.tensorflow.org/data/fsns-20160927/testdata/fsns-00000-of-00001')


In [ ]:
dataset = tf.data.TFRecordDataset(filenames = [fsns_test_file])
dataset

- 데이터 이해를 위해 `tf.data.Dataset` 외부에서 `tf.train.Example` 프로토콜로 작업할 수 있다.

In [ ]:
raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())

feature = parsed.features.feature
raw_img = feature['image/encoded'].bytes_list.value[0]
img = tf.image.decode_png(raw_img)
plt.imshow(img)
plt.axis('off')
plt.title(feature['image/text'].bytes_list.value[0])

In [ ]:
raw_example = next(iter(dataset))

In [ ]:
def tf_parse(eg):
  example = tf.io.parse_example(
      eg[tf.newaxis], {
          'image/encoded' : tf.io.FixedLenFeature(shape = (), dtype = tf.string),
          'image/text' : tf.io.FixedLenFeature(shape = (), dtype = tf.string)
      }
  )
  return example['image/encoded'][0], example['image/text'][0]

In [ ]:
img, txt = tf_parse(raw_example)
print(txt.numpy())
print(repr(img.numpy()[:20]), "...")

In [ ]:
decoded = dataset.map(tf_parse)
decoded

In [ ]:
image_batch, text_batch = next(iter(decoded.batch(10)))
image_batch.shape

### 시계열 윈도잉

In [ ]:
range_ds = tf.data.Dataset.range(100000) # 1e6으로 넣었는데 안됨

In [ ]:
# 시계열 데이터는 시간에 따라 데이터가 구성되어 있음 : 이를 batch 처리하는 게 제일 간단함
batches = range_ds.batch(10, drop_remainder = True)

for batch in batches.take(5):
  print(batch.numpy())

In [ ]:
# 미래 밀집 예측 : 기능, 레이블을 1단계씩 이동시킴
def dense_1_step(batch):
  return batch[:-1], batch[1:]

predict_dense_1_step = batches.map(dense_1_step)

for features, label in predict_dense_1_step.take(3):
  print(features.numpy(), " => ", label.numpy())

In [ ]:
# 전체 윈도우 예측 : batch를 2개로 분할함
batches = range_ds.batch(15, drop_remainder = True)

def label_next_5_steps(batch):
  return (batch[:-5], batch[-5:]) # Inputs(마지막 5스텝 제외), Labels(마지막 5스텝)

predict_5_steps = batches.map(label_next_5_steps)

for features, label in predict_5_steps.take(3):
  print(features.numpy(), ' => ', label.numpy())

In [ ]:
# 기능과 레이블 겹치기 : Dataset.zip
feature_length = 10
label_length = 3

features = range_ds.batch(feature_length, drop_remainder = True)
labels = range_ds.batch(feature_length).skip(1).map(lambda labels : labels[:label_length])

predicted_steps = tf.data.Dataset.zip((features, labels))

for features, label in predicted_steps.take(5):
  print(features.numpy(), ' -> ', label.numpy())

In [ ]:
# 보통은 window 쓰죠?
window_size = 5
windows = range_ds.window(window_size, shift = 1)
for sub_ds in windows.take(5):
  print(sub_ds)

In [ ]:
for x in windows.flat_map(lambda x: x).take(30):
  print(x.numpy(), end = ' ')

In [ ]:
# 거의 모든 경우 데이터세트를 batch 처리함
def sub_to_batch(sub):
  return sub.batch(window_size, drop_remainder=True)

for example in windows.flat_map(sub_to_batch).take(5):
  print(example.numpy())

In [ ]:
# shift 인수에 의해 윈도우 이동 정도가 제어된다.
def make_window_dataset(ds, window_size = 5, shift = 1, stride = 1):
  windows = ds.window(window_size, shift = shift, stride = stride)

  def sub_to_batch(sub):
    return sub.batch(window_size, drop_remainder = True)

  windows = windows.flat_map(sub_to_batch)
  return windows

In [ ]:
ds = make_window_dataset(range_ds, window_size = 10, shift = 5, stride = 3)

for example in ds.take(10):
  print(example.numpy())

In [ ]:
# 레이블 추출
dense_labels_ds = ds.map(dense_1_step)

for inputs, labels in dense_labels_ds.take(3):
  print(inputs.numpy(), '->', labels.numpy())

### 리샘플링
- 클래스 불균형이 매우 높은 데이터세트의 경우 데이터 세트를 다시 샘플링해아 함.
`tf.Data`는 2가지 방법을 제공한다.

In [ ]:
zip_path = tf.keras.utils.get_file(origin = 'https://storage.googleapis.com/download.tensorflow.org/data/creditcard.zip',
                                   fname = 'creditcard.zip',
                                   extract = True)

csv_path = zip_path.replace('.zip', '.csv')

In [ ]:
creditcard_ds = tf.data.experimental.make_csv_dataset(
    csv_path, batch_size = 1024, label_name = 'Class',
    column_defaults = [float()] * 30 + [int()]
)

In [ ]:
def count(counts, batch):
  features, labels = batch
  class_1 = labels == 1
  class_1 = tf.cast(class_1, tf.int32)

  class_0 = labels == 0
  class_0 = tf.cast(class_0, tf.int32)

  counts['class_0'] += tf.reduce_sum(class_0)
  counts['class_1'] += tf.reduce_sum(class_1)

  return counts

In [ ]:
counts = creditcard_ds.take(10).reduce(initial_state = {'class_0' : 0, 'class_1' : 0}, reduce_func = count)

counts = np.array([counts['class_0'].numpy(), counts['class_1'].numpy()]).astype(np.float32)

fractions = counts/counts.sum()

print(fractions)

#### 데이터 세트 리샘플링하기

In [ ]:
negative_ds = (creditcard_ds.unbatch().filter(lambda features, label : label == 0).repeat())
positive_ds = (creditcard_ds.unbatch().filter(lambda features, label : label == 1).repeat())

In [ ]:
for features, label in positive_ds.batch(10).take(1):
  print(label.numpy())

`tf.data.experimental.sample_from_dataset`를 쓰기 위해 데이터 세트와 각각의 가중치를 전달한다.

In [ ]:
balanced_ds = tf.data.experimental.sample_from_datasets([negative_ds, positive_ds], [0.5, 0.5]).batch(10)

In [ ]:
for features, labels in balanced_ds.take(10):
  print(labels.numpy())

`experimental.sample_from_datasets` 방식의 문제 : 개별 `tf.data.Dataset`이 필요하다는 점이다. `Dataset.filter`를 사용하면 되지만 모든 데이터가 2번 로드된다.
#### `data.experimental.rejection_resample` 이라는 게 있음
- 1번만 로드하면서 균형을 재조정함. 
- `class_func` 인수가 사용된다. 그냥 예제 보자.

In [ ]:
def class_func(features, label):
  return label

In [ ]:
# 이거!
resampler = tf.data.experimental.rejection_resample(
    class_func, target_dist=[.5, .5], initial_dist = fractions
)

In [ ]:
# 리샘플러는 개별 예시를 다루므로 unbatch 먼저 한다
resample_ds = creditcard_ds.unbatch().apply(resampler).batch(10)

In [ ]:
balanced_ds = resample_ds.map(lambda extra_label, features_and_label: features_and_label)

In [ ]:
for features, labels in balanced_ds.take(10):
  print(labels.numpy())

## 반복기 검사점 처리 `tf.train.Checkpoint`

In [ ]:
range_ds = tf.data.Dataset.range(20)

iterator = iter(range_ds)
ckpt = tf.train.Checkpoint(step = tf.Variable(0), iterator = iterator)
manager = tf.train.CheckpointManager(ckpt, '/tmp/my_ckpt', max_to_keep = 3)
print([next(iterator).numpy() for _ in range(5)])

save_path = manager.save()

print([next(iterator).numpy() for _ in range (5)])

ckpt.restore(manager.latest_checkpoint)

print([next(iterator).numpy() for _ in range(5)])

- `tf.py_function` 같은 외부 상태에 의존하는 반복기는 검사점 처리할 수 없다.

# 고급 API 사용하기

In [ ]:
train, test = tf.keras.datasets.fashion_mnist.load_data()

images, labels = train
images = images / 255.0
labels = labels.astype(np.int32)

In [ ]:
fmnist_train_ds = tf.data.Dataset.from_tensor_slices((images, labels))
fmnist_train_ds = fmnist_train_ds.shuffle(5000).batch(32)

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

In [ ]:
model.fit(fmnist_train_ds,epochs = 2)

- `Dataset.repeat()`를 호출해 무한한 데이터셋을 전달한다면 추가로 `steps_per_epoch`인수를 전달해줘야 한다

In [ ]:
model.fit(fmnist_train_ds.repeat(), epochs = 2, steps_per_epoch = 20)

In [ ]:
loss, accuracy = model.evaluate(fmnist_train_ds)
print("Loss : ", loss)
print("Accuracy : ", accuracy)

In [ ]:
predict_ds = tf.data.Dataset.from_tensor_slices(images).batch(32)
result = model.predict(predict_ds, steps = 10)
print(result.shape)

In [ ]:
result= model.predict(fmnist_train_ds, steps = 10)
print(result.shape)